In [1]:
import pandas as pd
# Helpful for showing indexing information
pd.set_option('display.max_colwidth', 150)

import os
import pyterrier as pt

import platform
platform.uname()

uname_result(system='Linux', node='gl1003.arc-ts.umich.edu', release='4.18.0-305.45.1.el8_4.x86_64', version='#1 SMP Wed Apr 6 13:48:37 EDT 2022', machine='x86_64')

In [2]:
if not pt.started():
    pt.init()

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
df_anime=pd.read_csv('data_processed/anime_with_review.csv')
df_anime=df_anime.rename(columns={'uid':'docno'})
df_anime.head(1)

,docno,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,wiki_link,anime_name,wiki_intro,review
0,28891,Haikyuu!! Second Season,"Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the...","['Comedy', 'Sports', 'Drama', 'School', 'Shounen']","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/76662.jpg,https://myanimelist.net/anime/28891/Haikyuu_Second_Season,http://en.wikipedia.org/wiki/Haiky%C5%AB!!,Haikyū!!,"Junior high school student, Shoyo Hinata, becomes obsessed with volleyball after catching a glimpse of Karasuno High School playing in Nationals o...","Overall: 10 \r\n \r\nPlot: 10 \r\n \r\nHaikyuu! is a sports genre anime, mainly tackling about Volleyball. In this kind of story, there's only one..."


In [4]:
df_anime['docno']=df_anime['docno'].astype(str)
df_anime['popularity']=df_anime['popularity'].astype(str)
df_anime['ranked']=df_anime['ranked'].astype(str)
df_anime['score']=df_anime['score'].astype(str)
df_anime['episodes']=df_anime['episodes'].astype(str)

In [5]:

pt_index_path = './pt_index'

if not os.path.exists(pt_index_path + "/data.properties"):

    indexer = pt.DFIndexer(pt_index_path,overwrite=True)
    index_ref = indexer.index(
    df_anime['synopsis'],df_anime['docno'],df_anime['title'],df_anime['popularity'],df_anime['ranked'],df_anime['score'],
    df_anime['episodes'],df_anime['wiki_intro'],df_anime['review']
)

else:
    # if you already have the index, create an IndexRef from the data in pt_index_path
    # that we can use to load using the IndexFactory
    # TODO
    index_ref = pt.IndexRef.of(pt_index_path+"/data.properties")
index = pt.IndexFactory.of(index_ref)

In [6]:
bm25_150 = pt.BatchRetrieve(index, wmodel="BM25")%150

In [13]:
query=pd.read_csv('data_processed/query_list.csv')
query.reset_index(inplace=True)
query.rename(columns={'Query':'query','index':'qid'},inplace=True)
train_query=query[query['Train/Test']=='Train'].drop(columns=['Train/Test'])
test_query=query[query['Train/Test']=='Test'].drop(columns=['Train/Test'])

In [14]:
train_query

,qid,query
0,0,"An adventure of revenge, brotherhood"
1,1,High school idol
2,2,Anime composed of independent stories
3,3,Anime featuring traditional Japanese culture
4,4,Romatic love story between a couple who struggle to be together
5,5,"Complicated relationship, the hero is liked by many female characters"
6,6,Seems to be a love story but actually not
7,7,"Sports anime, the main character has huge ambitions in mind"
8,8,Adapted from light novel
9,9,A group of people living together


In [15]:
result=bm25_150(train_query)
result=result.merge(df_anime,on=['docno'],how='left')
result=result[['qid','docno','rank','score_x','query','title','synopsis','genre','wiki_intro','review']]

In [16]:
result.rename(columns={'score_x':'bm25_score'})
result.to_csv('system_rating/baseline_rating.csv',index=False)

In [17]:
result_test=bm25_150(test_query)
result_test=result_test.merge(df_anime,on=['docno'],how='left')
result_test=result_test[['qid','docno','rank','score_x','query','title','synopsis','genre','wiki_intro','review']]
result_test.rename(columns={'score_x':'bm25_score'})
result_test.to_csv('system_rating/baseline_rating_test.csv',index=False)